# Linear Neural Network Classification

## Image Dataset

In the following sections, our discussion will center on the Fashion-MNIST dataset (Xiao et al., 2017) where consists of images representing 10 categories of clothing at the same pixel resolution.

In [ ]:
%matplotlib inline
import time
import torch
import torchvision
from torch import nn
from torch.nn import functional as F
from torchvision import transforms
from d2l import torch as d2l

d2l.use_svg_display()

### Loading the Dataset

In [ ]:

class FashionMNIST(d2l.DataModule):  # @save
    """The Fashion-MNIST dataset."""
    def __init__(self, batch_size=64, resize=(28, 28), root="./data", download=False):
        super().__init__()
        self.save_hyperparameters()
        self.root = root   # ✅ override default "../data"

        trans = transforms.Compose([
            transforms.Resize(resize),
            transforms.ToTensor()
        ])
        self.train = torchvision.datasets.FashionMNIST(
            root=self.root, train=True, transform=trans, download=download
        )
        self.val = torchvision.datasets.FashionMNIST(
            root=self.root, train=False, transform=trans, download=download
        )


Fashion-MNIST consists of images from 10 categories, each represented by 6000 images in the training dataset and by 1000 in the test dataset. 

In [ ]:
data = FashionMNIST(resize=(28, 28), root="/tmp/fashion_mnist", download=False)
len(data.train), len(data.val)

The images are grayscale and upscaled to  $32 \times 32$ pixels in resolution above. 

In [ ]:
data.train[2][1]#.shape


The Fashion-MNIST categories are labeled with names that are easily understood by humans.

In [ ]:
@d2l.add_to_class(FashionMNIST)  #@save
def text_labels(self, indices):
    """Return text labels."""
    labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
              'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    return [labels[int(i)] for i in indices]

### Minibatch

For convenience, we rely on the built-in data iterator to read from the training and test sets instead of implementing one manually.

In [ ]:
@d2l.add_to_class(FashionMNIST)  #@save
def get_dataloader(self, train):
    data = self.train if train else self.val
    return torch.utils.data.DataLoader(data, self.batch_size, shuffle=train,
                                       num_workers=self.num_workers)

In [ ]:
X, y = next(iter(data.train_dataloader()))
print(X.shape, X.dtype, y.shape, y.dtype)

To illustrate this, we load a minibatch of images using the `train_dataloader` method, which provides 64 images.

In [ ]:
X, y = next(iter(data.train_dataloader()))
print(X.shape, X.dtype, y.shape, y.dtype)

Let’s look at the time it takes to read the images. 

In [ ]:
tic = time.time()
for X, y in data.train_dataloader():
    continue
f'{time.time() - tic:.2f} sec'

### Visualization

The helper function `show_images` allows us to display the images along with their corresponding labels.

In [ ]:
def show_images(imgs, num_rows, num_cols, titles=None, scale=1.5):  #@save
    """Plot a list of images."""
    raise NotImplementedError

In [ ]:
@d2l.add_to_class(FashionMNIST)  #@save
def visualize(self, batch, nrows=1, ncols=8, labels=[]):
    X, y = batch
    if not labels:
        labels = self.text_labels(y)
    d2l.show_images(X.squeeze(1), nrows, ncols, titles=labels)
batch = next(iter(data.val_dataloader()))
data.visualize(batch)

## The Base Classification Model

This section introduces a base class for classification models, designed to streamline and simplify future implementations.

### The Classifier Class

In [ ]:
class Classifier(d2l.Module):  #@save
    """The base class of classification models."""
    def validation_step(self, batch):
        Y_hat = self(*batch[:-1])
        self.plot('loss', self.loss(Y_hat, batch[-1]), train=False)
        self.plot('acc', self.accuracy(Y_hat, batch[-1]), train=False)

In [ ]:
@d2l.add_to_class(d2l.Module)  #@save
def configure_optimizers(self):
    return torch.optim.SGD(self.parameters(), lr=self.lr)

### Accuracy

In classification tasks, we select the class with the highest predicted probability as the output, and accuracy is defined as the fraction of predictions that match the true labels.

In [ ]:
@d2l.add_to_class(Classifier)  #@save
def accuracy(self, Y_hat, Y, averaged=True):
    """Compute the number of correct predictions."""
    Y_hat = Y_hat.reshape((-1, Y_hat.shape[-1]))
    preds = Y_hat.argmax(axis=1).type(Y.dtype)
    compare = (preds == Y.reshape(-1)).type(torch.float32)
    return compare.mean() if averaged else compare

## Softmax Regression, Implementation from Scratch

### The Softmax

For a matrix $X$, we can sum either all elements by default or restrict the summation to a specific axis. The `axis` parameter allows us to calculate sums across rows or columns.

In [ ]:
X = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
X.sum(0, keepdims=True), X.sum(1, keepdims=True)

The softmax computation involves three steps: (i) exponentiating each element, (ii) summing the values in each row to obtain the normalization constant for that example, and (iii) dividing each row’s elements by its normalization constant so that the outputs sum to 1.

In [ ]:
def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1, keepdims=True)
    return X_exp / partition  # The broadcasting mechanism is applied here

For any input $X$, each element is transformed into a nonnegative value. Each row is then normalized so that its entries sum to 1, satisfying the requirements of a probability distribution.

In [ ]:
X = torch.rand((2, 5))
X_prob = softmax(X)
X_prob, X_prob.sum(1)

### The Model

we now implement the softmax regression model. In softmax regression, the network’s output size must match the number of classes. Therefore, with a dataset containing 10 classes, the model’s output dimension is set to 10.

In [ ]:
class SoftmaxRegressionScratch(d2l.Classifier):
    def __init__(self, num_inputs, num_outputs, lr, sigma=0.01):
        super().__init__()
        self.save_hyperparameters()
        self.W = torch.normal(0, sigma, size=(num_inputs, num_outputs),
                              requires_grad=True)
        self.b = torch.zeros(num_outputs, requires_grad=True)

    def parameters(self):
        return [self.W, self.b]

The code below defines how the network maps each input to an output. 

In [ ]:
@d2l.add_to_class(SoftmaxRegressionScratch)
def forward(self, X):
    X = X.reshape((-1, self.W.shape[0]))
    return softmax(torch.matmul(X, self.W) + self.b)

### The Cross-Entropy Loss

Next we need to implement the cross-entropy loss function.

Recall that cross-entropy takes the negative log-likelihood of the predicted probability as- signed to the true label. For efficiency we avoid Python for-loops and use indexing instead. In particular, the one-hot encoding in y allows us to select the matching terms in y_hat.

To see this in action we create sample data y_hat with 2 examples of predicted probabilities over 3 classes and their corresponding labels y. The correct labels are 0 and 2 respectively (i.e., the first and third class). Using y as the indices of the probabilities in y_hat, we can pick out terms efficiently.

In [ ]:
y = torch.tensor([0, 2])
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y_hat[[0, 1], y]

In [ ]:
def cross_entropy(y_hat, y, test=False):
    if test:
        print('print the highest probabilities', y_hat[list(range(len(y_hat))), y])
        print('and their logarithm', torch.log(y_hat[list(range(len(y_hat))), y]))
    return -torch.log(y_hat[list(range(len(y_hat))), y]).mean()

cross_entropy(y_hat, y, test=True)

In [ ]:
@d2l.add_to_class(SoftmaxRegressionScratch)
def loss(self, y_hat, y):
    return cross_entropy(y_hat, y)

### Training

We reuse the fit method defined in previous sections to train the model with 10 epochs.

In [ ]:
data = d2l.FashionMNIST(batch_size=256)
model = SoftmaxRegressionScratch(num_inputs=784, num_outputs=10, lr=0.1)
trainer = d2l.Trainer(max_epochs=10)
trainer.fit(model, data)

### Prediction

With training finished, the model is now prepared to perform image classification.

In [ ]:
X, y = next(iter(data.val_dataloader()))
preds = model(X).argmax(axis=1)
preds.shape

Our main focus is on the images that were misclassified. We display them by showing their true labels (top line of the text output) alongside the model’s predicted labels (bottom line).

In [ ]:
wrong = preds.type(y.dtype) != y
X, y, preds = X[wrong], y[wrong], preds[wrong]
labels = [a+'\n'+b for a, b in zip(
    data.text_labels(y), data.text_labels(preds))]
data.visualize([X, y], labels=labels)

## Softmax Regression, Concise Implementation

Similar to how high-level deep learning frameworks simplified the implementation of linear regression, they prove equally convenient in this context.

### Defining the Model

In [ ]:
class SoftmaxRegression(d2l.Classifier):  #@save
    """The softmax regression model."""
    def __init__(self, num_outputs, lr):
        super().__init__()
        self.save_hyperparameters()
        self.net = nn.Sequential(nn.Flatten(),
                                 nn.LazyLinear(num_outputs))

    def forward(self, X):
        return self.net(X)

### Softmax Revisited

Directly computing softmax followed by cross-entropy can cause numerical instability due to overflow or underflow in the exponentiation step. To address this, modern implementations combine softmax and cross-entropy into a single operation (using techniques like the LogSumExp trick), which ensures stability while still allowing us to compute probabilities when needed.

In [ ]:
@d2l.add_to_class(d2l.Classifier)  #@save
def loss(self, Y_hat, Y, averaged=True):
    Y_hat = Y_hat.reshape((-1, Y_hat.shape[-1]))
    Y = Y.reshape((-1,))
    return F.cross_entropy(
        Y_hat, Y, reduction='mean' if averaged else 'none')

### Training

In [ ]:
data = d2l.FashionMNIST(batch_size=256)
model = SoftmaxRegression(num_outputs=10, lr=0.1)
trainer = d2l.Trainer(max_epochs=10)
trainer.fit(model, data)

## Exercises

1. Does reducing the batch_size (for instance, to 1) affect the reading performance?

2. Deep learning uses many different number formats, including FP64 double precision (used extremely rarely), FP32 single precision, BFLOAT16 (good for compressed representations), FP16 (very unstable), TF32 (a new format from NVIDIA), and INT8. Compute the smallest and largest argument of the exponential function for which the result does not lead to numerical underflow or overflow.